In [ ]:
import pandas as pd
import pickle
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from modules.cleaners import simple_clean
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer
import re
import string
from gensim import matutils, models
import scipy.sparse

In [32]:
df_first = pd.read_pickle('pickled_data/data_first_clean.pkl')
df_second = pd.read_pickle('pickled_data/data_second_clean.pkl')
df_first.head()

,year,President,Party,speech,first_clean,first_clean_tokenized
0,1900,William McKinley,Republican,To the Senate and House of Representatives: \n...,to the senate and house of representatives: at...,"[senate, house, representatives, outgoing, old..."
1,1901,Theodore Roosevelt,Republican,To the Senate and House of Representatives: \n...,to the senate and house of representatives: th...,"[senate, house, representatives, congress, ass..."
2,1902,Theodore Roosevelt,Republican,To the Senate and House of Representatives: \n...,to the senate and house of representatives: we...,"[senate, house, representatives, still, contin..."
3,1903,Theodore Roosevelt,Republican,To the Senate and House of Representatives: \n...,to the senate and house of representatives: th...,"[senate, house, representatives, country, cong..."
4,1904,Theodore Roosevelt,Republican,To the Senate and House of Representatives: \n...,to the senate and house of representatives: th...,"[senate, house, representatives, nation, conti..."


In [33]:
def clean_for_tdm(text):
    '''Remove forward slash, punctuation and numbers'''
    text = text.replace("\\", "")
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text   

In [34]:
crisis = [1930, 1931, 1932, 1986, 1987, 1988, 2008, 2009, 2010]
growth = [1926, 1927, 1928, 1992, 1993, 1994, 2005, 2006, 2007]
selection = crisis+growth

In [35]:
clean_txt = lambda x: clean_for_tdm(x)

In [36]:
data = df_first[['year', 'first_clean']].copy()
data = data[data.year.isin(selection)]
data = data.reset_index(drop=True)
data['year'] = data['year'].apply(str)
data.to_pickle("pickled_data/for_word_cloud.pkl")
data

,year,first_clean
0,1926,members of the congress: in reporting to the c...
1,1927,members of the congress: it is gratifying to r...
2,1928,to the congress of the united states:no congre...
3,1930,to the senate and house of representatives: i ...
4,1931,to the senate and house of representatives: it...
5,1932,to the senate and house of representatives: in...
6,1986,"mr. speaker, mr. president, distinguished memb..."
7,1987,"thank you very much. mr. speaker, mr. presiden..."
8,1988,"""thank you. mr. speaker, mr. president, and di..."
9,1992,"mr. speaker and mr. president, distinguished m..."


In [37]:
data_clean = pd.DataFrame(data.first_clean.apply(clean_txt))
data_clean

,first_clean
0,members of the congress in reporting to the co...
1,members of the congress it is gratifying to re...
2,to the congress of the united statesno congres...
3,to the senate and house of representatives i h...
4,to the senate and house of representatives it ...
5,to the senate and house of representatives in ...
6,mr speaker mr president distinguished members ...
7,thank you very much mr speaker mr president di...
8,thank you mr speaker mr president and distingu...
9,mr speaker and mr president distinguished memb...


In [38]:
stop_words = text.ENGLISH_STOP_WORDS

cv = CountVectorizer(stop_words=stop_words)
data_cv = cv.fit_transform(data_clean['first_clean'])
data_tdm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_tdm.index = data.year

data_tdm

,abandon,abandoned,abandoning,abandonment,abbas,abess,abide,abiding,abidinglogic,abilities,...,youre,youth,youthful,youve,zarqawi,zeroemission,zimbabwe,zinc,zone,zones
year,,,,,,,,,,,,,,,,,,,,,
1926,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1927,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1928,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1930,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1931,1,1,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1932,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1986,0,2,0,1,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
1987,1,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
1988,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
data_tdm = data_tdm.transpose()
data_tdm.head()

year,1926,1927,1928,1930,1931,1932,1986,1987,1988,1992,1993,1994,2005,2006,2007,2008,2009,2010
abandon,0,0,0,0,1,0,0,1,0,0,0,0,2,1,1,1,0,0
abandoned,0,1,1,0,1,0,2,0,0,0,0,1,0,0,1,0,0,0
abandoning,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
abandonment,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
abbas,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [40]:
# Find the top 25 words in each speech
top_dict = {}
for c in data_tdm.columns:
    top = data_tdm[c].sort_values(ascending=False).head(25)
    top_dict[c]= list(zip(top.index, top.values))

top_dict

{'1926': [('government', 49),
  ('country', 36),
  ('national', 31),
  ('federal', 26),
  ('people', 25),
  ('present', 25),
  ('congress', 23),
  ('ought', 23),
  ('years', 22),
  ('reduction', 21),
  ('legislation', 21),
  ('general', 19),
  ('public', 19),
  ('time', 17),
  ('great', 17),
  ('agriculture', 17),
  ('war', 16),
  ('law', 15),
  ('work', 15),
  ('large', 14),
  ('nation', 14),
  ('year', 14),
  ('agricultural', 13),
  ('tariff', 13),
  ('development', 13)],
 '1927': [('government', 46),
  ('country', 25),
  ('congress', 25),
  ('public', 24),
  ('national', 23),
  ('people', 22),
  ('agriculture', 21),
  ('federal', 19),
  ('states', 19),
  ('department', 16),
  ('law', 15),
  ('land', 15),
  ('great', 15),
  ('farmer', 15),
  ('state', 15),
  ('property', 15),
  ('necessary', 14),
  ('legislation', 14),
  ('large', 14),
  ('order', 13),
  ('war', 13),
  ('provide', 13),
  ('years', 13),
  ('power', 13),
  ('reduction', 12)],
 '1928': [('government', 45),
  ('congress'

In [41]:
# Print the top 12 words used in each year
for year, top_words in top_dict.items():
    print(year)
    print(', '.join([word for word, count in top_words[0:11]]))
    print('---')

1926
government, country, national, federal, people, present, congress, ought, years, reduction, legislation
---
1927
government, country, congress, public, national, people, agriculture, federal, states, department, law
---
1928
government, congress, states, public, year, years, country, federal, new, present, private
---
1930
year, congress, government, construction, public, depression, federal, work, economic, employment, country
---
1931
banks, federal, government, action, year, meet, public, credit, congress, economic, financial
---
1932
government, economic, congress, federal, people, great, action, recovery, world, national, country
---
1986
america, future, people, american, work, applause, family, tonight, world, budget, government
---
1987
congress, people, america, applause, world, years, new, american, freedom, let, government
---
1988
america, applause, government, years, world, freedom, budget, people, lets, federal, family
---
1992
people, know, plan, world, right, help,

In [42]:
tdm = data.transpose()
tdm.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
year,1926,1927,1928,1930,1931,1932,1986,1987,1988,1992,1993,1994,2005,2006,2007,2008,2009,2010
first_clean,members of the congress: in reporting to the c...,members of the congress: it is gratifying to r...,to the congress of the united states:no congre...,to the senate and house of representatives: i ...,to the senate and house of representatives: it...,to the senate and house of representatives: in...,"mr. speaker, mr. president, distinguished memb...","thank you very much. mr. speaker, mr. presiden...","""thank you. mr. speaker, mr. president, and di...","mr. speaker and mr. president, distinguished m...","mr. president, mr. speaker, members of the hou...","thank you very much. mr. speaker, mr. presiden...","as a new congress gathers, all of us in the el...","mr. speaker, vice president cheney, members of...","thank you very much. and tonight, i have a hig...","the president: madam speaker, vice president c...","madame speaker, mr. vice president, members of...","madam speaker, vice president biden, members o..."


In [43]:
top_dict = {}
for c in data_tdm.columns:
    top = data_tdm[c].sort_values(ascending=False).head(30)
    top_dict[c]= list(zip(top.index, top.values))

top_dict

{'1926': [('government', 49),
  ('country', 36),
  ('national', 31),
  ('federal', 26),
  ('people', 25),
  ('present', 25),
  ('congress', 23),
  ('ought', 23),
  ('years', 22),
  ('reduction', 21),
  ('legislation', 21),
  ('general', 19),
  ('public', 19),
  ('time', 17),
  ('great', 17),
  ('agriculture', 17),
  ('war', 16),
  ('law', 15),
  ('work', 15),
  ('large', 14),
  ('nation', 14),
  ('year', 14),
  ('agricultural', 13),
  ('tariff', 13),
  ('development', 13),
  ('power', 13),
  ('tax', 12),
  ('states', 12),
  ('department', 12),
  ('good', 12)],
 '1927': [('government', 46),
  ('country', 25),
  ('congress', 25),
  ('public', 24),
  ('national', 23),
  ('people', 22),
  ('agriculture', 21),
  ('federal', 19),
  ('states', 19),
  ('department', 16),
  ('law', 15),
  ('land', 15),
  ('great', 15),
  ('farmer', 15),
  ('state', 15),
  ('property', 15),
  ('necessary', 14),
  ('legislation', 14),
  ('large', 14),
  ('order', 13),
  ('war', 13),
  ('provide', 13),
  ('years',

In [44]:
sparse_counts = scipy.sparse.csr_matrix(data_tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [45]:
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

In [46]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

[(0,
  '0.007*"people" + 0.007*"government" + 0.005*"country" + 0.005*"applause" + 0.005*"congress" + 0.005*"new" + 0.005*"american" + 0.004*"america" + 0.004*"years" + 0.004*"year"'),
 (1,
  '0.007*"people" + 0.006*"congress" + 0.006*"government" + 0.005*"year" + 0.005*"world" + 0.005*"america" + 0.004*"work" + 0.004*"federal" + 0.004*"economic" + 0.004*"years"')]

In [47]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)
lda.print_topics()

[(0,
  '0.007*"people" + 0.007*"america" + 0.006*"world" + 0.006*"government" + 0.006*"congress" + 0.005*"federal" + 0.005*"year" + 0.004*"american" + 0.004*"new" + 0.004*"nation"'),
 (1,
  '0.009*"government" + 0.006*"congress" + 0.005*"people" + 0.005*"country" + 0.004*"public" + 0.004*"years" + 0.004*"states" + 0.004*"national" + 0.004*"federal" + 0.003*"agriculture"'),
 (2,
  '0.009*"people" + 0.007*"applause" + 0.005*"work" + 0.005*"country" + 0.005*"year" + 0.005*"government" + 0.005*"new" + 0.005*"health" + 0.005*"american" + 0.005*"years"')]

In [48]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=10)
lda.print_topics()

[(0,
  '0.008*"people" + 0.008*"america" + 0.006*"new" + 0.006*"applause" + 0.005*"american" + 0.005*"world" + 0.005*"government" + 0.005*"congress" + 0.005*"years" + 0.005*"country"'),
 (1,
  '0.010*"government" + 0.007*"congress" + 0.006*"federal" + 0.005*"country" + 0.005*"public" + 0.005*"people" + 0.005*"national" + 0.004*"year" + 0.004*"states" + 0.004*"years"'),
 (2,
  '0.005*"american" + 0.005*"know" + 0.005*"economy" + 0.005*"plan" + 0.004*"people" + 0.004*"health" + 0.004*"new" + 0.004*"care" + 0.004*"america" + 0.003*"years"'),
 (3,
  '0.011*"people" + 0.010*"applause" + 0.006*"work" + 0.006*"know" + 0.006*"year" + 0.005*"health" + 0.005*"american" + 0.005*"care" + 0.005*"americans" + 0.005*"america"')]

In [49]:
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

In [50]:
data_nouns = pd.DataFrame(data_clean['first_clean'].apply(nouns))
data_nouns

,first_clean
0,members congress congress state union i peace ...
1,members congress year state union country whol...
2,congress statesno congress states state union ...
3,senate house representatives i honor requireme...
4,senate house representatives duty constitution...
5,senate house representatives accord duty trans...
6,mr speaker president members congress guests c...
7,thank mr speaker president members congress gu...
8,thank mr president members house senate years ...
9,mr speaker mr president members congress guest...


In [51]:
# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=stop_words)
data_cvn = cvn.fit_transform(data_nouns['first_clean'])
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names())
data_dtmn.index = data_nouns.index
data_dtmn

,abandonment,abbas,abilities,ability,abolition,abortion,abortions,absence,absent,absolute,...,york,youll,youth,youve,zarqawi,zeroemission,zimbabwe,zinc,zone,zones
0,0,0,0,3,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,4,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,1,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
4,1,0,1,3,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,1,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
8,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,1,0


In [52]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [53]:
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.009*"year" + 0.009*"world" + 0.009*"america" + 0.009*"years" + 0.008*"health" + 0.008*"country" + 0.008*"congress" + 0.008*"government" + 0.008*"americans" + 0.006*"nation"'),
 (1,
  '0.016*"government" + 0.010*"congress" + 0.009*"country" + 0.008*"year" + 0.006*"years" + 0.006*"states" + 0.005*"agriculture" + 0.005*"world" + 0.005*"legislation" + 0.005*"action"')]

In [54]:
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.015*"government" + 0.011*"congress" + 0.009*"country" + 0.008*"year" + 0.006*"action" + 0.006*"world" + 0.006*"banks" + 0.005*"years" + 0.005*"work" + 0.005*"nation"'),
 (1,
  '0.010*"health" + 0.010*"year" + 0.010*"america" + 0.009*"world" + 0.009*"country" + 0.009*"years" + 0.009*"americans" + 0.008*"care" + 0.008*"jobs" + 0.008*"government"'),
 (2,
  '0.016*"government" + 0.011*"congress" + 0.007*"years" + 0.007*"states" + 0.007*"country" + 0.006*"year" + 0.006*"world" + 0.005*"department" + 0.005*"agriculture" + 0.004*"legislation"')]

In [55]:
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.010*"years" + 0.009*"government" + 0.009*"world" + 0.009*"year" + 0.008*"plan" + 0.008*"america" + 0.008*"jobs" + 0.007*"country" + 0.007*"americans" + 0.007*"congress"'),
 (1,
  '0.010*"america" + 0.010*"health" + 0.009*"congress" + 0.009*"country" + 0.009*"year" + 0.009*"world" + 0.008*"security" + 0.008*"americans" + 0.008*"years" + 0.007*"government"'),
 (2,
  '0.014*"government" + 0.011*"congress" + 0.010*"year" + 0.009*"banks" + 0.008*"action" + 0.007*"construction" + 0.006*"world" + 0.006*"country" + 0.005*"employment" + 0.005*"depression"'),
 (3,
  '0.018*"government" + 0.011*"country" + 0.010*"congress" + 0.007*"years" + 0.007*"states" + 0.006*"legislation" + 0.006*"agriculture" + 0.006*"year" + 0.006*"law" + 0.006*"department"')]

In [56]:
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [57]:
data_nouns_adj = pd.DataFrame(data_clean['first_clean'].apply(nouns_adj))
data_nouns_adj

,first_clean
0,members congress congress state union i imposs...
1,members congress fourth consecutive year state...
2,congress united statesno congress united state...
3,senate house representatives i honor requireme...
4,senate house representatives duty constitution...
5,senate house representatives accord constituti...
6,mr speaker president members congress guests f...
7,thank much mr speaker president members congre...
8,thank speaker mr president members house senat...
9,mr speaker mr president members congress guest...


In [58]:
cvna = CountVectorizer(stop_words=stop_words, max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj['first_clean'])
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names())
data_dtmna.index = data_nouns_adj.index
data_dtmna

,abandonment,abbas,abiding,abidinglogic,abilities,ability,able,ablebodied,abolition,abortion,...,younger,youth,youthful,youve,zarqawi,zeroemission,zimbabwe,zinc,zone,zones
0,0,0,0,0,0,3,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,4,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,2,4,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,1,0,0,0,1,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
7,0,0,0,0,1,1,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [59]:
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [60]:
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.005*"agriculture" + 0.005*"present" + 0.004*"construction" + 0.004*"necessary" + 0.004*"banks" + 0.004*"department" + 0.004*"general" + 0.004*"reduction" + 0.003*"cent" + 0.003*"credit"'),
 (1,
  '0.008*"americans" + 0.006*"jobs" + 0.005*"plan" + 0.005*"applause" + 0.004*"tonight" + 0.004*"reform" + 0.003*"iraq" + 0.003*"energy" + 0.003*"spending" + 0.003*"terrorists"')]

In [61]:
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.009*"plan" + 0.008*"jobs" + 0.007*"americans" + 0.004*"reform" + 0.004*"workers" + 0.004*"investment" + 0.004*"banks" + 0.003*"tonight" + 0.003*"credit" + 0.003*"energy"'),
 (1,
  '0.005*"agriculture" + 0.005*"present" + 0.004*"department" + 0.004*"necessary" + 0.004*"general" + 0.004*"construction" + 0.004*"reduction" + 0.003*"cent" + 0.003*"relief" + 0.003*"foreign"'),
 (2,
  '0.008*"applause" + 0.007*"americans" + 0.004*"jobs" + 0.004*"tonight" + 0.004*"iraq" + 0.003*"agreement" + 0.003*"reform" + 0.003*"americas" + 0.003*"trade" + 0.003*"hope"')]

In [62]:
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.008*"banks" + 0.007*"construction" + 0.005*"employment" + 0.005*"depression" + 0.005*"financial" + 0.005*"credit" + 0.004*"recovery" + 0.004*"unemployment" + 0.004*"measures" + 0.004*"laws"'),
 (1,
  '0.000*"americans" + 0.000*"reduction" + 0.000*"credit" + 0.000*"present" + 0.000*"jobs" + 0.000*"banks" + 0.000*"general" + 0.000*"plan" + 0.000*"necessary" + 0.000*"reform"'),
 (2,
  '0.006*"present" + 0.005*"agriculture" + 0.005*"department" + 0.005*"general" + 0.004*"reduction" + 0.004*"necessary" + 0.004*"policy" + 0.003*"order" + 0.003*"land" + 0.003*"property"'),
 (3,
  '0.009*"americans" + 0.007*"jobs" + 0.006*"plan" + 0.005*"applause" + 0.005*"tonight" + 0.004*"reform" + 0.004*"iraq" + 0.003*"energy" + 0.003*"spending" + 0.003*"terrorists"')]

In [63]:
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=80)
ldana.print_topics()

[(0,
  '0.005*"agriculture" + 0.005*"present" + 0.005*"construction" + 0.004*"necessary" + 0.004*"banks" + 0.004*"general" + 0.004*"department" + 0.004*"reduction" + 0.004*"cent" + 0.003*"cooperation"'),
 (1,
  '0.008*"iraq" + 0.006*"americans" + 0.006*"terrorists" + 0.005*"applause" + 0.004*"iraqi" + 0.004*"terror" + 0.004*"hope" + 0.004*"tonight" + 0.003*"women" + 0.003*"goal"'),
 (2,
  '0.010*"jobs" + 0.009*"plan" + 0.009*"americans" + 0.005*"reform" + 0.004*"tonight" + 0.004*"businesses" + 0.004*"investment" + 0.004*"spending" + 0.004*"welfare" + 0.004*"applause"'),
 (3,
  '0.005*"agreement" + 0.004*"democratic" + 0.004*"americans" + 0.004*"applause" + 0.003*"revolution" + 0.003*"lets" + 0.003*"arms" + 0.003*"nuclear" + 0.003*"spending" + 0.003*"tonight"')]

In [66]:
# Let's take a look at which topics each transcript contains
corpus_transformed = ldana[corpusna]
print(len(corpus_transformed))
print(data_dtmna[:18])
print(len(data_dtmna.index))
#list(zip([a for [(a,b)] in corpus_transformed], data_dtmna.index))

18
    abandonment  abbas  abiding  abidinglogic  abilities  ability  able  \
0             0      0        0             0          0        3     1   
1             0      0        0             0          0        4     1   
2             1      0        0             0          0        2     4   
3             0      0        0             0          0        0     2   
4             1      0        0             0          1        3     0   
5             0      0        0             0          0        1     0   
6             1      0        0             1          0        0     0   
7             0      0        0             0          1        1     2   
8             0      0        0             0          0        0     0   
9             0      0        0             0          0        0     0   
10            0      0        1             0          0        0     3   
11            0      0        0             0          0        1     2   
12            0      1